In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from python.fluorophores import FlStatic, FlCollection
from python.psfs import PsfVectorial
from python.estimators import est_qLSQiter2D
from python.estimators import est_qLSQiter1D
from python.simulators import Simulator

## Vectorial PSF

In [ ]:
psf_vec = PsfVectorial()
psf_vec.setpinhole(AU=1)
psf_vec.zerooffset=0.000
sys_aberr = {}
sys_aberr['Zr'] = np.zeros((1,3))
sys_aberr['Zr'][0,0], sys_aberr['Zr'][0,1], sys_aberr['Zr'][0,2] = 4, 0, 0.0  # spherical aberrations 
sys_aberr['maskshift'] = [0,0]
psf_vec.setpar(**sys_aberr)

In [ ]:
fl = FlStatic(brightness=1000)  # define a static fluorophore
fl.pos = [10, 0, 0]

sim = Simulator(fluorophores=fl)

numberOfLocalizations=5000

In [ ]:
# define scan pattern
L = 75
zeroposx = np.atleast_2d(np.array([-1,1,0])*L/2)
probecenter = True  # should we also probe the center?
orbitpoints = 6
laserpowerdonut = 10  # relative, increases brightness
laserpowertophat = 80
laserpowerpf = 8
pointdwelltime = 0.1  # ms, measurement time in each point
repetitions = 1  # how often to repeat the pattern scan
sim.definePattern("donut", psf_vec, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerdonut, 
                  repetitions=repetitions)
                  
sim.definePattern("tophat_xy", psf_vec, 
                  phasemask="tophat",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowertophat, 
                  repetitions=repetitions,
                  dim=(0,1))

sim.definePattern("pf_x", psf_vec, 
                  phasemask="halfmoonx",
                  zeropos = zeroposx,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerpf, 
                  repetitions=repetitions,
                  dim=(0,))

sim.definePattern("pf_y", psf_vec, 
                  phasemask="halfmoony",
                  zeropos = zeroposx,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerpf, 
                  repetitions=repetitions,
                  dim=(1,))

In [ ]:
sim.defineComponent("estdonut", "estimator", est_qLSQiter2D, parameters=[L, probecenter], dim=(0,1))
sim.defineComponent("est_x", "estimator", est_qLSQiter1D, parameters=[L], dim=(0,))
sim.defineComponent("est_y", "estimator", est_qLSQiter1D, parameters=[L], dim=(1,))

## 2D donut

In [ ]:
seq = ["donut", "estdonut"]
out = sim.runSequence(seq, maxlocs=numberOfLocalizations)
print("no aberration donut:")
sim.summarize_results(out)
xs = np.arange(0, 70, 5)
displaywhat="rmse"
plt.figure()
_ = sim.scan_fov(seq, xs, clearfigure=True, tag="2D donut", ax1=displaywhat, linestyle='k')

## 3D donut

In [ ]:
seqthnoab = ["tophat_xy", "estdonut"]
out = sim.runSequence(seqthnoab, maxlocs=numberOfLocalizations)
print('no aberration tophat:')
sim.summarize_results(out)  # display summary of simulation
_ = sim.scan_fov(seqthnoab, xs, tag="3D donut", ax1=displaywhat, linestyle='r')

## PhaseFlux

In [ ]:
seqpf = ["pf_x", "est_x", "pf_y", "est_y"]
out = sim.runSequence(seqpf, maxlocs=numberOfLocalizations)
print('no aberration phaseflux:')
sim.summarize_results(out)  # display summary of simulation
_ = sim.scan_fov(seqpf, xs, tag="Half-moon", ax1=displaywhat, linestyle='b')

In [ ]:
psf2Ddonut, _ = psf_vec.imagestack("vortex")
psf3Ddonut, _ = psf_vec.imagestack("tophat")
psfPF, _ = psf_vec.imagestack("halfmoonx")

## Spherical Aberrations

In [ ]:
psf_veca = PsfVectorial()
psf_veca.setpinhole(AU=1)
psf_veca.zerooffset=0.000
sys_aberr = {}
sys_aberr['Zr'] = np.zeros((1,3))
sys_aberr['Zr'][0,0], sys_aberr['Zr'][0,1], sys_aberr['Zr'][0,2] = 4, 0, 0.15  # spherical aberrations 
sys_aberr['maskshift'] = [0,0]
psf_veca.setpar(**sys_aberr)

In [ ]:
sim.definePattern("donuta", psf_veca, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerdonut, 
                  repetitions=repetitions)
                  
sim.definePattern("tophat_xya", psf_veca, 
                  phasemask="tophat",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowertophat, 
                  repetitions=repetitions,
                  dim=(0,1))

sim.definePattern("pf_xa", psf_veca, 
                  phasemask="halfmoonx",
                  zeropos = zeroposx,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerpf, 
                  repetitions=repetitions,
                  dim=(0,))

sim.definePattern("pf_ya", psf_veca, 
                  phasemask="halfmoony",
                  zeropos = zeroposx,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerpf, 
                  repetitions=repetitions,
                  dim=(1,))

In [ ]:
seqdab = ["donuta", "estdonut"]
out = sim.runSequence(seqdab, maxlocs=numberOfLocalizations)

_ = sim.scan_fov(seqdab, xs, tag="2D donut spherical", ax1=displaywhat, linestyle='k-.')

In [ ]:
seqthab = ["tophat_xya", "estdonut"]
out = sim.runSequence(seqthab, maxlocs=numberOfLocalizations)

_ = sim.scan_fov(seqthab, xs, tag="3D donut spherical", ax1=displaywhat, linestyle='r-.')

In [ ]:
seqpfa = ["pf_xa", "est_x", "pf_ya", "est_y"]
out = sim.runSequence(seqpfa, maxlocs=numberOfLocalizations)
_ = sim.scan_fov(seqpfa, xs, tag="Half-moon spherical", ax1=displaywhat, linestyle='b-.')

In [ ]:
psf2Ddonuta, _ = psf_veca.imagestack("vortex")
psf3Ddonuta, _ = psf_veca.imagestack("tophat")
psfPFa, _ = psf_veca.imagestack("halfmoonx")

## Vertical coma aberration

In [ ]:
psf_vecca = PsfVectorial()
psf_vecca.setpinhole(AU=1)
psf_vecca.zerooffset=0.000
sys_aberr = {}

# vertical coma aberrations 
# sys_aberr['Zr'] = np.zeros((1,3))
# sys_aberr['Zr'][0,0], sys_aberr['Zr'][0,1], sys_aberr['Zr'][0,2] = 3, -1, 0.15

# 45-degree coma aberrations 
sys_aberr['Zr'] = np.zeros((2,3))
sys_aberr['Zr'][0,0], sys_aberr['Zr'][0,1], sys_aberr['Zr'][0,2] = 3, -1, 0.075
sys_aberr['Zr'][1,0], sys_aberr['Zr'][1,1], sys_aberr['Zr'][1,2] = 3, 1, 0.075

sys_aberr['maskshift'] = [0,0]
psf_vecca.setpar(**sys_aberr)

In [ ]:
sim.definePattern("donutca", psf_vecca, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerdonut, 
                  repetitions=repetitions)
                  
sim.definePattern("tophat_xyca", psf_vecca, 
                  phasemask="tophat",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowertophat, 
                  repetitions=repetitions,
                  dim=(0,1))

sim.definePattern("pf_xca", psf_vecca, 
                  phasemask="halfmoonx",
                  zeropos = zeroposx,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerpf, 
                  repetitions=repetitions,
                  dim=(0,))

sim.definePattern("pf_yca", psf_vecca, 
                  phasemask="halfmoony",
                  zeropos = zeroposx,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerpf, 
                  repetitions=repetitions,
                  dim=(1,))

In [ ]:
seqdcab = ["donutca", "estdonut"]
out = sim.runSequence(seqdcab, maxlocs=numberOfLocalizations)

_ = sim.scan_fov(seqdcab, xs, tag="2D donut coma", ax1=displaywhat, linestyle='k--')

In [ ]:
seqthcab = ["tophat_xyca", "estdonut"]
out = sim.runSequence(seqthcab, maxlocs=numberOfLocalizations)

_ = sim.scan_fov(seqthcab, xs, tag="3D donut coma", ax1=displaywhat, linestyle='r--')

In [ ]:
seqpfca = ["pf_xca", "est_x", "pf_yca", "est_y"]
out = sim.runSequence(seqpfca, maxlocs=numberOfLocalizations)
_ = sim.scan_fov(seqpfca, xs, tag="Half-moon coma", ax1=displaywhat, linestyle='b--')

In [ ]:
psf2Ddonutca, _ = psf_vecca.imagestack("vortex")
psf3Ddonutca, _ = psf_vecca.imagestack("tophat")
psfPFca, _ = psf_vecca.imagestack("halfmoonx")

## Oblique astigmatismmatism aberration

In [ ]:
psf_vecaa = PsfVectorial()
psf_vecaa.setpinhole(AU=1)
psf_vecaa.zerooffset=0.000
sys_aberr = {}

# oblique astigmatismmatism aberrations 
# sys_aberr['Zr'] = np.zeros((1,3))
# sys_aberr['Zr'][0,0], sys_aberr['Zr'][0,1], sys_aberr['Zr'][0,2] = 2, -2, 0.15

# 45-degree astigmatismmatism aberrations 
sys_aberr['Zr'] = np.zeros((2,3))
sys_aberr['Zr'][0,0], sys_aberr['Zr'][0,1], sys_aberr['Zr'][0,2] = 2, -2, 0.075
sys_aberr['Zr'][1,0], sys_aberr['Zr'][1,1], sys_aberr['Zr'][1,2] = 2, 2, 0.075


sys_aberr['maskshift'] = [0,0]
psf_vecaa.setpar(**sys_aberr)

In [ ]:
sim.definePattern("donutaa", psf_vecaa, 
                  phasemask="vortex",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerdonut, 
                  repetitions=repetitions)
                  
sim.definePattern("tophat_xyaa", psf_vecaa, 
                  phasemask="tophat",
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowertophat, 
                  repetitions=repetitions,
                  dim=(0,1))

sim.definePattern("pf_xaa", psf_vecaa, 
                  phasemask="halfmoonx",
                  zeropos = zeroposx,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerpf, 
                  repetitions=repetitions,
                  dim=(0,))

sim.definePattern("pf_yaa", psf_vecaa, 
                  phasemask="halfmoony",
                  zeropos = zeroposx,
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpowerpf, 
                  repetitions=repetitions,
                  dim=(1,))

In [ ]:
seqdaab = ["donutaa", "estdonut"]
out = sim.runSequence(seqdaab, maxlocs=numberOfLocalizations)

_ = sim.scan_fov(seqdaab, xs, tag="2D donut astigmatism", ax1=displaywhat, linestyle='k:')

In [ ]:
seqthaab = ["tophat_xyaa", "estdonut"]
out = sim.runSequence(seqthaab, maxlocs=numberOfLocalizations)

_ = sim.scan_fov(seqthaab, xs, tag="3D donut astigmatism", ax1=displaywhat, linestyle='r:')

In [ ]:
seqpfaa = ["pf_xaa", "est_x", "pf_yaa", "est_y"]
out = sim.runSequence(seqpfaa, maxlocs=numberOfLocalizations)
_ = sim.scan_fov(seqpfaa, xs, tag="Half-moon astigmatism", ax1=displaywhat, linestyle='b:')

In [ ]:
psf2Ddonutaa, _ = psf_vecaa.imagestack("vortex")
psf3Ddonutaa, _ = psf_vecaa.imagestack("tophat")
psfPFaa, _ = psf_vecaa.imagestack("halfmoonx")

In [ ]:
# Fix plot and save
axh = plt.gca()
axh.set_ybound(lower=0)
axh.set_xbound(lower=0)
axf = plt.gcf()
axf.tight_layout()
axf.savefig("example2b_compare_PSFs_FOV.pdf")

## Multiple fluorophores

In [ ]:
fc = FlCollection()
fc.add(fl)
fl2 = FlStatic()
fl2.pos = [0, 0, 700]
fc.add(fl2)
fl3 = FlStatic()
fl3.pos = [50, 350, 900]
fc.add(fl3)
sim.fluorophores = fc

In [ ]:
seq = ["donut", "estdonut"]
out = sim.runSequence(seq, maxlocs=numberOfLocalizations)
print("fl donut:")
sim.summarize_results(out)
_ = sim.scan_fov(seq, xs, tag="2D donut fl", ax1=displaywhat, linestyle='k--')

In [ ]:
seqthnoab = ["tophat_xy", "estdonut"]
out = sim.runSequence(seqthnoab, maxlocs=numberOfLocalizations)
print('fl tophat:')
sim.summarize_results(out)  # display summary of simulation
_ = sim.scan_fov(seqthnoab, xs, tag="3D donut fl", ax1=displaywhat, linestyle='r--')

In [ ]:
out = sim.runSequence(seqpf, maxlocs=numberOfLocalizations)
print('fl phaseflux:')
sim.summarize_results(out)  # display summary of simulation
_ = sim.scan_fov(seqpf, xs, tag="Half-moon fl", ax1=displaywhat, linestyle='b--')

## Background

In [ ]:
sim.fluorophores = fl
sim.background = 3
seq = ["donut", "estdonut"]
out = sim.runSequence(seq, maxlocs=numberOfLocalizations)
print("bg donut:")
sim.summarize_results(out)
print(np.mean(out.bg_photons_gt))
_ = sim.scan_fov(seq, xs, tag="bg donut:", ax1=displaywhat, linestyle='k:')

In [ ]:
seqthnoab = ["tophat_xy", "estdonut"]
out = sim.runSequence(seqthnoab, maxlocs=numberOfLocalizations)
print('bg tophat:')
sim.summarize_results(out)  # display summary of simulation
print(np.mean(out.bg_photons_gt))
sim.scan_fov(seqthnoab, xs, tag="bg tophat:", ax1=displaywhat, linestyle='r:')

In [ ]:
out = sim.runSequence(seqpf, maxlocs=numberOfLocalizations)
print('bg phaseflux:')
sim.summarize_results(out)  # display summary of simulation
print(np.mean(out.bg_photons_gt))
sim.scan_fov(seqpf, xs, tag="Half-moon bg", ax1=displaywhat, linestyle='b:')

In [ ]:
plt.gca().set_ylim(0, L/2)

# plot PSFs

In [ ]:
from python.tools import colormaps

midp = int(np.ceil(psf2Ddonut.shape[2]/2))

pa2d=np.vstack([psf2Ddonut[:,:,midp], psf2Ddonuta[:,:,midp], psf2Ddonutca[:,:,midp], psf2Ddonutaa[:,:,midp]])
pa3d=np.vstack([psf3Ddonut[:,:,midp], psf3Ddonuta[:,:,midp], psf3Ddonutca[:,:,midp], psf3Ddonutaa[:,:,midp]])
papf=np.vstack([psfPF[:,:,midp], psfPFa[:,:,midp], psfPFca[:,:,midp], psfPFaa[:,:,midp]])

fig, ax = plt.subplots()
ax.imshow(np.hstack([pa2d/np.max(pa2d), pa3d/np.max(pa3d), papf/np.max(papf)]), cmap=colormaps.parula)
ax.set_aspect('equal')
ax.axis('off')

fig.tight_layout()
fig.savefig("abberations.pdf")